##### Copyright 2019 Google LLC.

***Licensed under the Apache License, Version 2.0 (the "License");***

# **AI BEATS by Metawave**

Our plan is to help artists reach artistic growth by providing them with an AI-built tool to recommend and provide useful information for improvement. The tool is comprised of the Autotune module, the Bass modifier, a tool for finding corresponding instruments, and a tool for automatic music creation, including overlaying, crossfading, and mixing the tunes.

* [GANSynth ICLR paper](https://arxiv.org/abs/1809.11096)
* [Audio Examples](http://goo.gl/magenta/gansynth-examples) 


<img src="https://storage.googleapis.com/magentadata/papers/gansynth/figures/models.jpeg" alt="GANSynth figure" width="600">

## 1: Environment Setup


This notebook synthesizes audio from uploaded MIDI files. There are two different flavors:
* Interpolate between random instruments
* Interpolate between two chosen instruments


Have fun! And please feel free to hack this notebook to make your own creative interactions.

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Then press the **Play** button on the left of each of the cells
* Double-click any of the cells to view the code


## 2: Installation

**blabla**

In [ ]:
#@title Install { vertical-output: true }

#@markdown Install all the prerequisities. Shouldn't take more than a minute. Hang in there! :)

print("Started installing. \n\n.")
!pip install note_seq
!pip install visual_midi
!pip install audioop
!pip install pydub
!pip install midi2audio


import note_seq, numpy as np, os
from google.colab import files
import audioop
import string
import glob, shutil
import os
from sys import builtin_module_names
import math
import array
import argparse
import numpy as np
import pydub
from pydub.playback import play
import IPython

from audioop import lin2adpcm
from concurrent.futures.thread import BrokenThreadPool
from hashlib import sha384
from tqdm import tqdm
import audioop
from pydub import AudioSegment

print("\n\nYeaa baby. Let's get it!")

Started installing. 

.
ERROR: Could not find a version that satisfies the requirement audioop (from versions: none)
ERROR: No matching distribution found for audioop


Yeaa baby. Let's get it!


## Uploading your favourite music ## 

In [ ]:
#@title Upload your own music

#@markdown Upload an MP3 or MIDI file for audio synthesis, or use the provided default.


# Helper functions
def load_midi(midi_path, min_pitch=36, max_pitch=84):
  """Load midi as a notesequence."""

  ns = note_seq.midi_file_to_sequence_proto(midi_path)
  pitches = np.array([n.pitch for n in ns.notes])
  velocities = np.array([n.velocity for n in ns.notes])
  start_times = np.array([n.start_time for n in ns.notes])
  end_times = np.array([n.end_time for n in ns.notes])
  valid = np.logical_and(pitches >= min_pitch, pitches <= max_pitch)
  notes = {'pitches': pitches[valid],
           'velocities': velocities[valid],
           'start_times': start_times[valid],
           'end_times': end_times[valid]}
  return ns, notes

from pydub.playback import play
import IPython
def play_song(song, filename = 'sample.mp3'):
  song.export(filename)
  IPython.display.Audio(filename)

ARRAY_RANGES = {
    8: (-0x80, 0x7f),
    16: (-0x8000, 0x7fff),
    32: (-0x80000000, 0x7fffffff),
}

def get_min_max_value(bit_depth):
    return ARRAY_RANGES[bit_depth]

FRAME_WIDTHS = {
    8: 1,
    16: 2,
    32: 4,
}


def get_frame_width(bit_depth):
    return FRAME_WIDTHS[bit_depth]

MIDI_SONG_DEFAULT = 'metawave_sample.mp3'

def upload():
  '''Upload a .wav file.'''
  filemap = files.upload()
  file_list = []
  for key, value in filemap.items():
    fname = os.path.join('/content/gansynth/midi', key)
    with open(fname, 'wb') as f:
      f.write(value)
      print('Writing {}'.format(fname))
    file_list.append(fname)
  return file_list



filename = "Upload you own" #@param ["c.mp3", "Upload you own"]

from midi2audio import FluidSynth
fs = FluidSynth()

if filename == 'Upload your own':
  try:
    f = upload()
  except Exception as e:
    print('Upload Cancelled')
  if f.endswith('mid'):
    fs.midi_to_audio(f, 'c.mp3')

  audio = AudioSegment.from_mp3(f)

play = False #@param {type:"boolean"}
if play: play_song(audio)

In [ ]:
print(type(audio))

<class 'pydub.audio_segment.AudioSegment'>


In [ ]:
## manjka: design, music visualization, beats and instruments integration, help artists = input song, output recommendation
## vse pejt čez

In [ ]:
## note: your song when uploaded is noted under the argument "song", if you wanna go back to a previous version, just reclick this cell. 
song = audio

## Emphasizing the Bass ## 

In [ ]:
#@title Emphasize Bass Section 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


## offer an explanation, play song after every remake
ATTENUATE_DB = 49 #@param {type:"slider", min:0, max:50, step:1}
ACCENTUATE_DB = 21 #@param {type:"slider", min:0, max:50, step:1}



def db_to_float(db, using_amplitude=True):
    """
    Converts the input db to a float, which represents the equivalent
    ratio in power.
    """
    db = float(db)
    if using_amplitude:
        return 10 ** (db / 20)
    else:  # using power
        return 10 ** (db / 10)
        
def bass_line_freq(track):
    sample_track = list(track)

    # c-value
    est_mean = np.mean(sample_track)

    # a-value
    est_std = 3 * np.std(sample_track) / (math.sqrt(2))

    bass_factor = int(round((est_std - est_mean) * 0.005))

    return bass_factor

def emphasise_bass(song):
    filtered = song.low_pass_filter(bass_line_freq(song.get_array_of_samples()))
    combined = (song - ATTENUATE_DB).overlay(filtered + ACCENTUATE_DB)
    return combined

song = emphasise_bass(song)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

import time
filename = 'sample.mp3'
if play: 
  song.export(filename, format = 'mp3')
  time.sleep(5)
  IPython.display.Audio(filename)
if download: download_mp3(song, "song.mp3")

In [ ]:
#@title Download your newly made Audio Track
#@markdown Get your song! 

# Write the file

def download_mp3(audio, file):
  song.export(file)

file = '' #@param {type:"string"}
song.export(file)

## Volume Mixer

In [ ]:
#@title Play With the Volume 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


def song_length(filename):
    song = AudioSegment.from_mp3(filename)
    return int(song.duration_seconds)



## keep in mind, negative volume
volume = 5 #@param {type:"slider", min:0, max:50, step:1}
lower_interval = 178 #@param {type:"slider", min:0, max:300, step:1}
upper_interval = 19 #@param {type:"slider", min:0, max:300, step:1}

## if interval je zuni pesmi vrži warning

if lower_interval > upper_interval:
  print("You have set the upper_interval lower than the lower_interval. ")

def volume_up(song, volume = 20, down = 0, up = 4):
    down, up = down*1000, up*1000
    segment = song[down:up].apply_gain(volume)
    seg1, seg3 = song[:down], song[up:]

    return seg1.append(segment, crossfade = 1000).append(seg3, crossfade = 1000)

song = volume_up(song, volume = volume, down = lower_interval, up = upper_interval)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

You have set the upper_interval lower than the lower_interval. 


## Apply Basses ## 

In [ ]:
#@title Apply Basses 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.

bass_volume = 5 #@param {type:"slider", min:0, max:100, step:1}
def apply_basses(song, bass_volume = 10):
    files = glob.iglob('**/*.wav')
    basses = [AudioSegment.from_wav(b).apply_gain(bass_volume) for b in files]
    for clip in basses:
        song = song.overlay(clip, position=6000, times = 2)

song = apply_basses(song, bass_volume = bass_volume)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

In [ ]:
#@title Download your newly made 8D Audio Track
#@markdown Get your song! 

# Write the file

def download_mp3(audio, file):
  song.export(file)

file = 'track.mp3' #@param {type:"string"}
song.export(file)

## Low pass filter ## 

In [ ]:
#@title Apply Low Pass Filter 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.

cutoff = 5 #@param {type:"slider", min:0, max:250, step:1}
play = True #@param ["False", "True"] {type:"raw"}
download = False #@param ["False", "True"] {type:"raw"}


frame_rate = 60

def low_pass_filter(seg, cutoff):
    """
        cutoff - Frequency (in Hz) where higher frequency signal will begin to
            be reduced by 6dB per octave (doubling in frequency) above this point
    """
    RC = 1.0 / (cutoff * 2 * math.pi)
    dt = 1.0 / frame_rate

    alpha = dt / (RC + dt)
    
    original = seg.get_array_of_samples()
    filteredArray = array.array(seg.array_type, original)
    
    frame_count = int(seg.frame_count())

    last_val = [0] * seg.channels
    for i in range(seg.channels):
        last_val[i] = filteredArray[i] = original[i]

    for i in range(1, frame_count):
        for j in range(seg.channels):
            offset = (i * seg.channels) + j
            last_val[j] = last_val[j] + (alpha * (original[offset] - last_val[j]))
            filteredArray[offset] = int(last_val[j])

    return seg._spawn(data=filteredArray)

song = low_pass_filter(song, cutoff)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

## High Pass Filter

In [ ]:
#@title Apply High Pass Filter 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


cutoff = 5 #@param {type:"slider", min:0, max:250, step:1}

def high_pass_filter(seg, cutoff):
    """
        cutoff - Frequency (in Hz) where lower frequency signal will begin to
            be reduced by 6dB per octave (doubling in frequency) below this point
    """
    RC = 1.0 / (cutoff * 2 * math.pi)
    dt = 1.0 / frame_rate

    alpha = RC / (RC + dt)

    minval, maxval = get_min_max_value(seg.sample_width * 8)
    
    original = seg.get_array_of_samples()
    filteredArray = array.array(seg.array_type, original)
    
    frame_count = int(seg.frame_count())

    last_val = [0] * seg.channels
    for i in range(seg.channels):
        last_val[i] = filteredArray[i] = original[i]

    for i in range(1, frame_count):
        for j in range(seg.channels):
            offset = (i * seg.channels) + j
            offset_minus_1 = ((i-1) * seg.channels) + j

            last_val[j] = alpha * (last_val[j] + original[offset] - original[offset_minus_1])
            filteredArray[offset] = int(min(max(last_val[j], minval), maxval))

    return seg._spawn(data=filteredArray)

song = high_pass_filter(song, cutoff)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

## Apply Gain Stereo ## 

In [ ]:
#@title Apply Gain Stereo 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


left_gain = 5 #@param {type:"slider", min:0, max:50, step:1}
right_gain = 7 #@param {type:"slider", min:0, max:50, step:1}


def apply_gain_stereo(seg, left_gain=0.0, right_gain=0.0):
    """
    left_gain - amount of gain to apply to the left channel (in dB)
    right_gain - amount of gain to apply to the right channel (in dB)
    
    note: mono audio segments will be converted to stereo
    """
    if seg.channels == 1:
        left = right = seg
    elif seg.channels == 2:
        left, right = seg.split_to_mono()
    
    l_mult_factor = db_to_float(left_gain)
    r_mult_factor = db_to_float(right_gain)
    
    left_data = audioop.mul(left._data, left.sample_width, l_mult_factor)
    left_data = audioop.tostereo(left_data, left.sample_width, 1, 0)
    
    right_data = audioop.mul(right._data, right.sample_width, r_mult_factor)
    right_data = audioop.tostereo(right_data, right.sample_width, 0, 1)
    
    output = audioop.add(left_data, right_data, seg.sample_width)
    
    return seg._spawn(data=output,
                overrides={'channels': 2,
                           'frame_width': 2 * seg.sample_width})
    
song = apply_gain_stereo(song, left_gain = left_gain, right_gain = right_gain)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

AttributeError: ignored

In [ ]:
#@title Download your newly made Audio Track
#@markdown Get your song! 

# Write the file

def download_mp3(audio, file):
  song.export(file)

file = 'track.mp3' #@param {type:"string"}
song.export(file)

## Speed Up ## 

In [ ]:
#@title Speed Up Or Slow Down 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


speed = -0.4 #@param {type:"slider", min:-5, max:5, step:0.1}


def speedup(seg, playback_speed=1.5, chunk_size=150, crossfade=25):
    atk = 1.0 / playback_speed

    if playback_speed < 2.0:
        # throwing out more than half the audio - keep 50ms chunks
        ms_to_remove_per_chunk = int(chunk_size * (1 - atk) / atk)
    else:
        # throwing out less than half the audio - throw out 50ms chunks
        ms_to_remove_per_chunk = int(chunk_size)
        chunk_size = int(atk * chunk_size / (1 - atk))

    # the crossfade cannot be longer than the amount of audio we're removing
    crossfade = min(crossfade, ms_to_remove_per_chunk - 1)

    # DEBUG
    #print("chunk: {0}, rm: {1}".format(chunk_size, ms_to_remove_per_chunk))

    chunks = make_chunks(seg, chunk_size + ms_to_remove_per_chunk)
    if len(chunks) < 2:
        raise Exception("Could not speed up AudioSegment, it was too short {2:0.2f}s for the current settings:\n{0}ms chunks at {1:0.1f}x speedup".format(
            chunk_size, playback_speed, seg.duration_seconds))

    # we'll actually truncate a bit less than we calculated to make up for the
    # crossfade between chunks
    ms_to_remove_per_chunk -= crossfade

    # we don't want to truncate the last chunk since it is not guaranteed to be
    # the full chunk length
    last_chunk = chunks[-1]
    chunks = [chunk[:-ms_to_remove_per_chunk] for chunk in chunks[:-1]]

    out = chunks[0]
    for chunk in chunks[1:]:
        out = out.append(chunk, crossfade=crossfade)

    out += last_chunk
    return out

song = speedup(song, speed)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")

TypeError: ignored

## Cut, cut, cut ##

In [ ]:
#@title Cut the Song 🎵
#@markdown Select the number of seconds to take in interpolating between each random instrument. Larger numbers will have slower and smoother interpolations.


on_the_left_side = 0 #@param {type:"slider", min:0, max:20, step:0.1}
on_the_right_side = 0 #@param {type:"slider", min:0, max:20, step:0.1}


def make_chunks(audio_segment, chunk_length):
    number_of_chunks = math.ceil(len(audio_segment) / float(chunk_length))
    return [audio_segment[i * chunk_length:(i + 1) * chunk_length]
            for i in range(int(number_of_chunks))]

def cut_song(song, on_the_left_side, on_the_right_side):
  song = song[on_the_left_side:]
  song = song[:on_the_right_side]
  return song

song = cut_song(song, on_the_left_side, on_the_right_side)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}

if play: play_song(song)
if download: download_mp3(song, "song.mp3")


In [ ]:
#@title Download your newly made 8D Audio Track
#@markdown Get your song! 

# Write the file

def download_mp3(audio, file):
  song.export(file)

file = 'track.mp3' #@param {type:"string"}
song.export(file)

##  **8D AUDIO !!!**

In [ ]:
#@title Transform any song into 8D Audio 🎵


def transformto8D(song):
    adjust_jump, segment_length = 8, 200

    trap_song = song[0]
    pan_limit=[]
    limit_left  = -100

    for i in range(100):
        if int(limit_left) >= 100:
            break
        pan_limit.append(limit_left)
        limit_left+=adjust_jump

    pan_limit.append(100)

    for i in range(0,len(pan_limit)): pan_limit[i] = pan_limit[i] /100
    c=0
    flag = True

    for i in range(0,len(song)-segment_length, segment_length):

        peice = song[i:i+segment_length]

        if c==0 and not flag:
            flag =True
            c=c+2

        if c==len(pan_limit):
            c = c-2
            flag =False

        if flag:
            panned = peice.pan(pan_limit[c])
            c+=1

        else:
            panned = peice.pan(pan_limit[c])
            c-=1
        trap_song =trap_song+panned
        
    return trap_song

song = transformto8D(song)

play = True #@param {type:"boolean"}
download = False #@param {type:"boolean"}
if play: play_song(song)
if download: download_mp3(song, "song.mp3")

In [ ]:
#@title Download your newly made 8D Audio Track
#@markdown Get your song! 

# Write the file

def download_mp3(audio, file):
  song.export(file)

file = 'track.mp3' #@param {type:"string"}
song.export(file)

AttributeError: ignored